In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import fnmatch
import errno    
import glob
from textwrap import dedent
from riboraptor.helpers import mkdir_p
from riboraptor.sradb import SRAdb
from riboraptor.utils import copy_sra_data, create_config_file

re_ribo_root_dir = '/staging/as/skchoudh/SRA_datasets/'
samples_to_process_dir = '/staging/as/skchoudh/re-ribo-datasets/'
re_ribo_config_dir = '/home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs'
out_dir = '/staging/as/skchoudh/re-ribo-analysis'

datasets = {}
datasets['hg38'] = pd.read_excel('../data/datasets/re-ribo-datasets.xlsx', sheet_name='hg38')
datasets['mm10'] = pd.read_excel('../data/datasets/re-ribo-datasets.xlsx', sheet_name='mm10')

Populating the interactive namespace from numpy and matplotlib


/home/cmb-panasas2/skchoudh/software_frozen/anaconda27/envs/riboraptor/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/cmb-panasas2/skchoudh/software_frozen/anaconda27/envs/riboraptor/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
datasets['hg38'].sort_values(by=['Tissue'])

,Publication,SRP,Status,PMID,Tissue
8,Loayza-Puch et al. (2013),SRP020544,NaN,23594524.0,BJ fibroblast
10,Rooijers et al. (2013),SRP027514,NaN,24301020.0,BJ fibroblast
3,Gonzalez et al. (2014),SRP031501,NaN,25122893.0,Brain
6,Lee et al. (2012),SRP014629,NaN,22927429.0,HEK293
7,Liu et al. (2012),SRP017263,NaN,23290916.0,HEK293
9,Reid and Nicchitta (2012),SRP007963,NaN,22199352.0,HEK293
0,Andreev et al. (2014),SRP038695,Done,25621764.0,HEK293T
4,Guo et al. (2010),SRP002605,NaN,20703300.0,HeLa
11,Stadler & Fire (2011),SRP010374,NaN,22045228.0,HeLa
13,Stumpf et al. (2013),SRP029589,NaN,24120665.0,HeLa


In [3]:
datasets['mm10'].sort_values(by=['Tissue'])

,Publication,SRP,Status,PMID,Tissue
6,Eichhorn et al. (2014),SRP045501,NaN,25263593.0,3T3/HEK293
7,Fields et al. (2015),SRP064972,NaN,26638175.0,Bone marrow-derived dendritic cells
15,Ingolia et al. (2011),SRP007567,NaN,22056041.0,E14
18,Laguesse et al. (2015),SRP065804,NaN,26651292.0,E14.5 forebrain
0,AlvarezDominguez et al. (2017),SRP077522,Started,27899360.0,E14.5 mouse fetal livers
16,Ingolia et al. (2014),SRP045257,NaN,25159147.0,ESC
14,Hurt et al. (2013),SRP016625,NaN,23766421.0,ESC
24,You et al. (2015),SRP064129,NaN,26443847.0,ESC
5,Cho et al. (2015),SRP062407,NaN,26430118.0,Hippocampal tissue
11,Guo et al. (2010),SRP003554,NaN,20703300.0,Neutrophils from bone marrow


In [4]:
!cat /staging/as/skchoudh/wen_list2.txt

SRP007567
SRP010679
SRP011987
SRP017942
SRP027015
SRP027514
SRP028325
SRP029589
SRP031501
SRP031849
SRP042937
SRP043153
SRP045214
SRP048708
SRP055009
SRP063852
SRP092068
SRP098789
SRP109126


In [19]:
! cat /staging/as/skchoudh/drop.txt

SRP033366
SRP028243


In [5]:
db = SRAdb('/staging/as/skchoudh/SRAmetadb.sqlite')


In [21]:
srp = 'SRP033366'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df

100%|██████████| 54/54 [00:01<00:00, 42.66it/s]


Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/BDGP6_SRP033366.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP033366,SRX384536,SRR1039726,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,965031242,25395559,None,38.0
1,SRP033366,SRX384537,SRR1039727,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1034182046,27215317,None,38.0
2,SRP033366,SRX384533,SRR1039728,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1175770616,30941332,None,38.0
3,SRP033366,SRX384538,SRR1039729,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1055604736,27779072,None,38.0
4,SRP033366,SRX384539,SRR1039730,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1001443412,26353774,None,38.0
5,SRP033366,SRX384540,SRR1039731,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,962270162,25322899,None,38.0
6,SRP033366,SRX384541,SRR1039732,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1130344048,29745896,None,38.0
7,SRP033366,SRX384563,SRR1039754,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,803300430,21139485,None,38.0
8,SRP033366,SRX384528,SRR1039719,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,871736640,21793416,None,40.0
9,SRP033366,SRX384529,SRR1039720,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,992204600,24805115,None,40.0


In [23]:
srp = 'SRP028243'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df

100%|██████████| 14/14 [00:00<00:00, 67.67it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/BDGP6_SRP028243.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP028243,SRX327686,SRR942868,7227,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2742897550,54857951,None,50.0
1,SRP028243,SRX327687,SRR942870,7227,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,3330922750,66618455,None,50.0
2,SRP028243,SRX327688,SRR942872,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,4447400350,88948007,None,50.0
3,SRP028243,SRX327689,SRR942873,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,4021570750,80431415,None,50.0
4,SRP028243,SRX327690,SRR942874,7227,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,1386531100,27730622,None,50.0
5,SRP028243,SRX327691,SRR942875,7227,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,1404463000,28089260,None,50.0
6,SRP028243,SRX327692,SRR942876,7227,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,1611862050,32237241,None,50.0
7,SRP028243,SRX327693,SRR942877,7227,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2009793400,40195868,None,50.0
8,SRP028243,SRX327694,SRR942878,7227,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,4594046100,91880922,None,50.0
9,SRP028243,SRX327695,SRR942879,7227,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,3372299400,67445988,None,50.0


In [18]:
srp = 'SRP109126'
df = db.sra_convert(srp)
df = df[df.library_layout.str.contains('SINGLE')]
copy_sra_data(df)
create_config_file(df=df)
df

100%|██████████| 12/12 [00:00<00:00, 35.28it/s]


Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/hg38_SRP109126.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP109126,SRX2916238,SRR5680913,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,11171423928,219047528,None,51.0
1,SRP109126,SRX2916239,SRR5680914,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,12429551649,243716699,None,51.0
2,SRP109126,SRX2916240,SRR5680915,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,6311924373,123763223,None,51.0
3,SRP109126,SRX2916241,SRR5680916,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,5240288760,102750760,None,51.0
4,SRP109126,SRX2916242,SRR5680917,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,7556499864,148166664,None,51.0
5,SRP109126,SRX2916243,SRR5680918,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,19177226703,376024053,None,51.0
6,SRP109126,SRX2916244,SRR5680919,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,13797550149,270540199,None,51.0
7,SRP109126,SRX2916245,SRR5680920,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,41151859149,806899199,None,51.0
8,SRP109126,SRX2916246,SRR5680921,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,19411035336,380608536,None,51.0
9,SRP109126,SRX2916247,SRR5680922,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,25865692491,507170441,None,51.0


# srp = 'SRP055009'
df = db.sra_convert(srp)
df = df[df.library_layout.str.contains('SINGLE')]
copy_sra_data(df)
create_config_file(df=df)
df

In [6]:
srp = 'SRP042937'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df

100%|██████████| 2/2 [00:00<00:00, 28.86it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/hg38_SRP042937.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP042937,SRX569214,SRR1333393,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,7750346150,155006923,None,50.0
1,SRP042937,SRX569215,SRR1333394,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,7947884750,158957695,None,50.0


In [11]:
srp = 'SRP098789'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df

100%|██████████| 26/26 [00:00<00:00, 75.92it/s]


Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/hg38_SRP098789.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP098789,SRX2536403,SRR5227288,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2104142750,42082855,None,50.0
1,SRP098789,SRX2536404,SRR5227289,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2082873050,41657461,None,50.0
2,SRP098789,SRX2536405,SRR5227290,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2023148650,40462973,None,50.0
3,SRP098789,SRX2536406,SRR5227291,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2057165950,41143319,None,50.0
4,SRP098789,SRX2536407,SRR5227292,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,3027621850,60552437,None,50.0
5,SRP098789,SRX2536408,SRR5227293,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2135456900,42709138,None,50.0
6,SRP098789,SRX2536409,SRR5227294,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,3800106100,76002122,None,50.0
7,SRP098789,SRX2536410,SRR5227295,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2306857400,46137148,None,50.0
8,SRP098789,SRX2536411,SRR5227296,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2636889200,52737784,None,50.0
9,SRP098789,SRX2536412,SRR5227297,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,3019068250,60381365,None,50.0


In [10]:
srp = 'SRP092068'
df = db.sra_convert(srp)
df.taxon_id = 9606
df['library_strategy'] = 'SINGLE'
copy_sra_data(df)
create_config_file(df=df)
df

100%|██████████| 14/14 [00:00<00:00, 50.68it/s]


Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/hg38_SRP092068.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP092068,SRX2268396,SRR4450319,9606,None,None,SINGLE,None,None,1082460850,21649217,None,50.0
1,SRP092068,SRX2268397,SRR4450320,9606,None,None,SINGLE,None,None,892411700,17848234,None,50.0
2,SRP092068,SRX2268398,SRR4450321,9606,None,None,SINGLE,None,None,938725900,18774518,None,50.0
3,SRP092068,SRX2268399,SRR4450322,9606,None,None,SINGLE,None,None,903547100,18070942,None,50.0
4,SRP092068,SRX2268400,SRR4450323,9606,None,None,SINGLE,None,None,1026896450,20537929,None,50.0
5,SRP092068,SRX2268401,SRR4450324,9606,None,None,SINGLE,None,None,786340550,15726811,None,50.0
6,SRP092068,SRX2268402,SRR4450325,9606,None,None,SINGLE,None,None,965706550,19314131,None,50.0
7,SRP092068,SRX2268403,SRR4450326,9606,None,None,SINGLE,None,None,1078819900,21576398,None,50.0
8,SRP092068,SRX2268404,SRR4450327,9606,None,None,SINGLE,None,None,866353000,17327060,None,50.0
9,SRP092068,SRX2268405,SRR4450328,9606,None,None,SINGLE,None,None,853020650,17060413,None,50.0


In [29]:
srp = 'SRP045214'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df

100%|██████████| 36/36 [00:00<00:00, 245.01it/s]


Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/hg38_SRP045214.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP045214,SRX668702,SRR1535516,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1388693222,28340678,None,49.0
1,SRP045214,SRX668702,SRR1535517,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1424468612,29070788,None,49.0
2,SRP045214,SRX668702,SRR1535518,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1425256287,29086863,None,49.0
3,SRP045214,SRX668703,SRR1535519,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,683628155,13951595,None,49.0
4,SRP045214,SRX668703,SRR1535520,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,690055485,14082765,None,49.0
5,SRP045214,SRX668703,SRR1535521,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,691138924,14104876,None,49.0
6,SRP045214,SRX668704,SRR1535522,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1205642109,24604941,None,49.0
7,SRP045214,SRX668704,SRR1535523,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1317078987,26879163,None,49.0
8,SRP045214,SRX668704,SRR1535524,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1317779295,26893455,None,49.0
9,SRP045214,SRX668705,SRR1535525,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1009215417,20596233,None,49.0


In [27]:
srp = 'SRP048708'
df = db.sra_convert(srp)
#df = df[df.library_layout.str.contains('SINGLE')]
copy_sra_data(df)

create_config_file(df=df)
df

100%|██████████| 18/18 [00:00<00:00, 238.55it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP048708.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP048708,SRX726845,SRR1605293,10090,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,1459053694,29776606,None,49.0
1,SRP048708,SRX726846,SRR1605294,10090,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,966776272,19730128,None,49.0
2,SRP048708,SRX726847,SRR1605295,10090,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,1414627452,28869948,None,49.0
3,SRP048708,SRX726848,SRR1605296,10090,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,1440599608,29399992,None,49.0
4,SRP048708,SRX726849,SRR1605297,10090,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,1136222143,23188207,None,49.0
5,SRP048708,SRX726850,SRR1605298,10090,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,984695964,20095836,None,49.0
6,SRP048708,SRX726851,SRR1605299,10090,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,1302987763,26591587,None,49.0
7,SRP048708,SRX726852,SRR1605300,10090,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,1254700272,25606128,None,49.0
8,SRP048708,SRX726856,SRR1605304,10090,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,1201536938,24521162,None,49.0
9,SRP048708,SRX726857,SRR1605305,10090,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,1323491862,27010038,None,49.0


In [24]:
srp = 'SRP055009'
df = db.sra_convert(srp)
df = df[df.library_layout.str.contains('SINGLE')]
#copy_sra_data(df)
df
#create_config_file(df=df)


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP055009,SRX876724,SRR1803147,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,5739317496,159425486,None,36.0
1,SRP055009,SRX876726,SRR1803149,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2914746408,80965178,None,36.0
2,SRP055009,SRX876727,SRR1803150,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,3528403740,98011215,None,36.0
3,SRP055009,SRX876731,SRR1803154,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,5136706692,142686297,None,36.0
4,SRP055009,SRX876732,SRR1803155,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,8108226756,225228521,None,36.0
5,SRP055009,SRX876733,SRR1803156,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,4466315664,124064324,None,36.0
6,SRP055009,SRX876734,SRR1803157,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,3736954836,103804301,None,36.0
7,SRP055009,SRX876735,SRR1803158,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,3532569444,98126929,None,36.0
8,SRP055009,SRX876736,SRR1803159,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,3674943288,102081758,None,36.0
9,SRP055009,SRX876737,SRR1803160,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,4429868004,123051889,None,36.0


In [19]:
srp = 'SRP010679'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df

100%|██████████| 12/12 [00:00<00:00, 24.66it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/hg38_SRP010679.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP010679,SRX118285,SRR403882,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM869036: mRNA-seq for vehicle treated PC3 ce...,886121960,22153049,None,40.0
1,SRP010679,SRX118286,SRR403883,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM869037: Footprint for vehicle treated PC3 c...,830504160,20762604,None,40.0
2,SRP010679,SRX118287,SRR403884,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM869038: mRNA-seq for rapamcyin treated PC3 ...,875152240,21878806,None,40.0
3,SRP010679,SRX118288,SRR403885,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM869039: Footprint for rapamcyin treated PC3...,789711800,19742795,None,40.0
4,SRP010679,SRX118289,SRR403886,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM869040: mRNA-seq for PP242 treated PC3 cell...,786234960,19655874,None,40.0
5,SRP010679,SRX118290,SRR403887,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM869041: Footprint for PP242 treated PC3 cel...,788273800,19706845,None,40.0
6,SRP010679,SRX118291,SRR403888,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM869042: mRNA-seq for vehicle treated PC3 ce...,979031600,24475790,None,40.0
7,SRP010679,SRX118292,SRR403889,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM869043: Footprint for vehicle treated PC3 c...,975917920,24397948,None,40.0
8,SRP010679,SRX118293,SRR403890,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM869044: mRNA-seq for rapamcyin treated PC3 ...,764698520,19117463,None,40.0
9,SRP010679,SRX118294,SRR403891,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM869045: Footprint for rapamcyin treated PC3...,1017802720,25445068,None,40.0


In [ ]:
srp = 'SRP010679'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df

In [18]:
srp = 'SRP007567'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df

100%|██████████| 37/37 [00:00<00:00, 54.57it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP007567.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP007567,SRX084813,SRR315598,10090,other,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM765290: embryoid body 8d in low-adhesion di...,623205135,17805861,None,35.0
1,SRP007567,SRX084824,SRR315625,10090,other,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,"GSM765301: ES cell feeder-free, w/ LIF 60 s CY...",603997170,17257062,None,35.0
2,SRP007567,SRX084808,SRR315591,10090,other,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM765285: embryoid body 8d in low-adhesion di...,326863200,8171580,None,40.0
3,SRP007567,SRX084810,SRR315593,10090,other,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM765287: ES cell 36 hours in low-adhesion di...,430345080,10758627,None,40.0
4,SRP007567,SRX084812,SRR315595,10090,other,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,"GSM765289: ES cell feeder-free, w/ LIF 60 s CY...",470525440,11763136,None,40.0
5,SRP007567,SRX084813,SRR315597,10090,other,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM765290: embryoid body 8d in low-adhesion di...,432360800,10809020,None,40.0
6,SRP007567,SRX084813,SRR315599,10090,other,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM765290: embryoid body 8d in low-adhesion di...,563503120,14087578,None,40.0
7,SRP007567,SRX084815,SRR315601,10090,other,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,"GSM765292: ES cell feeder-free, w/ LIF 60 s CY...",716602880,17915072,None,40.0
8,SRP007567,SRX084815,SRR315602,10090,other,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,"GSM765292: ES cell feeder-free, w/ LIF 60 s CY...",979899000,24497475,None,40.0
9,SRP007567,SRX084816,SRR315603,10090,other,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,"GSM765293: ES cell feeder-free, w/ LIF 60 s em...",769056720,19226418,None,40.0


In [15]:
srp = 'SRP034750'
df = db.sra_convert(srp)
#copy_sra_data(df)
#create_config_file(df=df)
df

,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP034750,SRX399799,SRR1062197,7955,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,304000000,4000000,None,76.0
1,SRP034750,SRX399799,SRR1062198,7955,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,215212696,2831746,None,76.0
2,SRP034750,SRX399799,SRR1062199,7955,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,304000000,4000000,None,76.0
3,SRP034750,SRX399799,SRR1062200,7955,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,304000000,4000000,None,76.0
4,SRP034750,SRX399799,SRR1062201,7955,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,304000000,4000000,None,76.0
5,SRP034750,SRX399799,SRR1062202,7955,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,304000000,4000000,None,76.0
6,SRP034750,SRX399799,SRR1062203,7955,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,304000000,4000000,None,76.0
7,SRP034750,SRX399799,SRR1062204,7955,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,304000000,4000000,None,76.0
8,SRP034750,SRX399799,SRR1062205,7955,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,304000000,4000000,None,76.0
9,SRP034750,SRX399799,SRR1062206,7955,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,304000000,4000000,None,76.0


In [ ]:
srp = 'SRP010679'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df

In [12]:
srp = 'SRP045214'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df

100%|██████████| 36/36 [00:00<00:00, 118.90it/s]


Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/hg38_SRP045214.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP045214,SRX668702,SRR1535516,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1388693222,28340678,None,49.0
1,SRP045214,SRX668702,SRR1535517,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1424468612,29070788,None,49.0
2,SRP045214,SRX668702,SRR1535518,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1425256287,29086863,None,49.0
3,SRP045214,SRX668703,SRR1535519,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,683628155,13951595,None,49.0
4,SRP045214,SRX668703,SRR1535520,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,690055485,14082765,None,49.0
5,SRP045214,SRX668703,SRR1535521,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,691138924,14104876,None,49.0
6,SRP045214,SRX668704,SRR1535522,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1205642109,24604941,None,49.0
7,SRP045214,SRX668704,SRR1535523,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1317078987,26879163,None,49.0
8,SRP045214,SRX668704,SRR1535524,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1317779295,26893455,None,49.0
9,SRP045214,SRX668705,SRR1535525,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1009215417,20596233,None,49.0


In [10]:
srp = 'SRP055707'
df = db.sra_convert(srp)
#copy_sra_data(df)
#create_config_file(df=df)
df

,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP055707,SRX894007,SRR1822445,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,2588329650,51766593,None,50.0
1,SRP055707,SRX894008,SRR1822446,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,3450906650,69018133,None,50.0
2,SRP055707,SRX894009,SRR1822447,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,2983328950,59666579,None,50.0
3,SRP055707,SRX894010,SRR1822448,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,3659227900,73184558,None,50.0
4,SRP055707,SRX894011,SRR1822449,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,867903050,17358061,None,50.0
5,SRP055707,SRX894012,SRR1822450,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1254858150,25097163,None,50.0
6,SRP055707,SRX894013,SRR1822451,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,893037500,17860750,None,50.0
7,SRP055707,SRX894014,SRR1822452,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1287696900,25753938,None,50.0
8,SRP055707,SRX894015,SRR1822453,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1156349950,23126999,None,50.0
9,SRP055707,SRX894016,SRR1822454,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1275071300,25501426,None,50.0


In [9]:
srp = 'SRP056647'
df = db.sra_convert(srp)
df = df[df.taxon_id==4932]
copy_sra_data(df)
create_config_file(df=df)
df


100%|██████████| 90/90 [00:00<00:00, 267.51it/s]


Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/sacCerR64_SRP056647.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP056647,SRX971707,SRR1944937,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,588362136,20126894,None,29.232635
1,SRP056647,SRX971683,SRR1944913,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1095851943,37334344,None,29.352382
2,SRP056647,SRX971682,SRR1944912,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,855004779,28905052,None,29.579770
3,SRP056647,SRX971708,SRR1944938,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,889343727,29893433,None,29.750472
4,SRP056647,SRX971706,SRR1944936,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,680664501,22825808,None,29.819952
5,SRP056647,SRX971685,SRR1944915,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,970021648,32467816,None,29.876406
6,SRP056647,SRX971761,SRR1944993,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,449713788,14979681,None,30.021586
7,SRP056647,SRX971684,SRR1944914,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,525959616,17505437,None,30.045500
8,SRP056647,SRX971747,SRR1944979,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1216625176,40491503,None,30.046432
9,SRP056647,SRX971755,SRR1944987,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,155111036,5157425,None,30.075287


In [8]:
srp = 'SRP038695'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df


100%|██████████| 12/12 [00:00<00:00, 28.17it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/hg38_SRP038695.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP038695,SRX474109,SRR1173905,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,794041050,24061850,None,33.0
1,SRP038695,SRX474110,SRR1173906,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,285938268,8664796,None,33.0
2,SRP038695,SRX474111,SRR1173907,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,783682548,23747956,None,33.0
3,SRP038695,SRX474112,SRR1173908,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,268895682,8148354,None,33.0
4,SRP038695,SRX474113,SRR1173909,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,4573442293,93335557,None,49.0
5,SRP038695,SRX474114,SRR1173911,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1281109508,26145092,None,49.0
6,SRP038695,SRX474115,SRR1173913,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,5910143428,120615172,None,49.0
7,SRP038695,SRX474116,SRR1173915,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1433175177,29248473,None,49.0
8,SRP038695,SRX474113,SRR1173910,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,645383400,8605112,None,75.0
9,SRP038695,SRX474114,SRR1173912,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,294934875,3932465,None,75.0


In [6]:
srp = 'SRP002605'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df


100%|██████████| 24/24 [00:00<00:00, 77.16it/s]


Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/hg38_SRP002605.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP002605,SRX021966,SRR057511,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546920: Footprint_mock_32hr_runs1-2,173069316,4807481,None,36.0
1,SRP002605,SRX021966,SRR057512,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546920: Footprint_mock_32hr_runs1-2,670274352,18618732,None,36.0
2,SRP002605,SRX021967,SRR057513,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546921: mRNASeq_mock_32hr_runs1-3,159276276,4424341,None,36.0
3,SRP002605,SRX021967,SRR057515,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546921: mRNASeq_mock_32hr_runs1-3,92929104,2581364,None,36.0
4,SRP002605,SRX021968,SRR057516,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546922: Footprint_miR-155_32hr_runs1-2,212446440,5901290,None,36.0
5,SRP002605,SRX021968,SRR057517,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546922: Footprint_miR-155_32hr_runs1-2,522725688,14520158,None,36.0
6,SRP002605,SRX021969,SRR057518,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546923: mRNASeq_miR-155_32hr_runs1-3,216374004,6010389,None,36.0
7,SRP002605,SRX021969,SRR057519,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546923: mRNASeq_miR-155_32hr_runs1-3,207243972,5756777,None,36.0
8,SRP002605,SRX021969,SRR057520,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546923: mRNASeq_miR-155_32hr_runs1-3,94523436,2625651,None,36.0
9,SRP002605,SRX021970,SRR057521,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546924: Footprint_miR-1_32hr_runs1-2,212789088,5910808,None,36.0


In [7]:
srp = 'SRP003554'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df


100%|██████████| 10/10 [00:00<00:00, 96.11it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP003554.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP003554,SRX026871,SRR065774,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546987: Footprint_wild-type_runs1-2,220030992,6111972,None,36.0
1,SRP003554,SRX026871,SRR065775,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546987: Footprint_wild-type_runs1-2,466656444,12962679,None,36.0
2,SRP003554,SRX026872,SRR065776,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546988: mRNASeq_wild-type_runs1-3,205130160,5698060,None,36.0
3,SRP003554,SRX026872,SRR065777,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546988: mRNASeq_wild-type_runs1-3,428629788,11906383,None,36.0
4,SRP003554,SRX026872,SRR065778,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546988: mRNASeq_wild-type_runs1-3,62993088,1749808,None,36.0
5,SRP003554,SRX026873,SRR065779,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546989: Footprint_mir-223-knockout_runs1-2,211817628,5883823,None,36.0
6,SRP003554,SRX026873,SRR065780,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546989: Footprint_mir-223-knockout_runs1-2,549140364,15253899,None,36.0
7,SRP003554,SRX026874,SRR065781,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546990: mRNASeq_mir-223-knockout_runs1-3,171575640,4765990,None,36.0
8,SRP003554,SRX026874,SRR065782,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546990: mRNASeq_mir-223-knockout_runs1-3,490349304,13620814,None,36.0
9,SRP003554,SRX026874,SRR065783,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546990: mRNASeq_mir-223-knockout_runs1-3,97069104,2696364,None,36.0


In [6]:
srp = 'SRP049168'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 12/12 [00:00<00:00, 66.27it/s]


Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/hg38_SRP049168.py
Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP049168.py


In [7]:
srp = 'SRP074765'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 82/82 [00:00<00:00, 70.03it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP074765.py


In [10]:
srp = 'SRP064304'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 884/884 [00:03<00:00, 251.67it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP064304.py


In [11]:
srp = 'SRP078005'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 18/18 [00:00<00:00, 61.26it/s]


Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP078005.py


In [18]:
srp = 'SRP021058'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 12/12 [00:00<00:00, 144.59it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP021058.py


In [15]:
srp = 'SRP076093'
df = db.sra_convert(srp)
df

,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP076093,SRX1818687,SRR3623562,10090,size fractionation,SINGLE -,ncRNA-Seq,TRANSCRIPTOMIC,None,1273893500,25477870,None,50.0
1,SRP076093,SRX1818688,SRR3623563,10090,size fractionation,SINGLE -,ncRNA-Seq,TRANSCRIPTOMIC,None,1327949700,26558994,None,50.0
2,SRP076093,SRX1818689,SRR3623564,10090,size fractionation,SINGLE -,ncRNA-Seq,TRANSCRIPTOMIC,None,1501223250,30024465,None,50.0
3,SRP076093,SRX1818682,SRR3623557,1247190,size fractionation,SINGLE -,ncRNA-Seq,TRANSCRIPTOMIC,None,1290162950,25803259,None,50.0
4,SRP076093,SRX1818683,SRR3623558,1247190,size fractionation,SINGLE -,ncRNA-Seq,TRANSCRIPTOMIC,None,961952850,19239057,None,50.0
5,SRP076093,SRX1818684,SRR3623559,1247190,size fractionation,SINGLE -,ncRNA-Seq,TRANSCRIPTOMIC,None,1061998750,21239975,None,50.0
6,SRP076093,SRX1818685,SRR3623560,1247190,size fractionation,SINGLE -,ncRNA-Seq,TRANSCRIPTOMIC,None,1201295750,24025915,None,50.0
7,SRP076093,SRX1818686,SRR3623561,1247190,size fractionation,SINGLE -,ncRNA-Seq,TRANSCRIPTOMIC,None,1108448800,22168976,None,50.0


In [17]:
srp = 'SRP076093'
df = db.sra_convert(srp)
#copy_sra_data(df)
create_config_file(df=df)

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP076093.py


KeyError: 1247190

In [14]:
srp = 'SRP056576'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 56/56 [00:00<00:00, 64.65it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP056576.py


In [16]:
srp = 'SRP030014'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 19/19 [00:00<00:00, 55.40it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP030014.py


In [23]:
srp = 'SRP043036'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df



100%|██████████| 12/12 [00:00<00:00, 108.21it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/sacCerR64_SRP043036.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP043036,SRX581792,SRR1363414,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1957041936,54362276,None,36.0
1,SRP043036,SRX581797,SRR1363419,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1435095972,39863777,None,36.0
2,SRP043036,SRX581798,SRR1363420,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1078644888,29962358,None,36.0
3,SRP043036,SRX581798,SRR1363421,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1430761104,39743364,None,36.0
4,SRP043036,SRX581799,SRR1363422,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1487820708,41328353,None,36.0
5,SRP043036,SRX581799,SRR1363423,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1635037848,45417718,None,36.0
6,SRP043036,SRX581790,SRR1363412,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1734374300,34687486,None,50.0
7,SRP043036,SRX581791,SRR1363413,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1791908950,35838179,None,50.0
8,SRP043036,SRX581793,SRR1363415,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1855938900,37118778,None,50.0
9,SRP043036,SRX581795,SRR1363417,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1708898250,34177965,None,50.0


In [25]:
srp = 'SRP063562'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df



100%|██████████| 5/5 [00:00<00:00, 71.09it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/MG1655_SRP063562.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP063562,SRX1219745,SRR2340141,511145,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1967578900,39351578,None,50.0
1,SRP063562,SRX1219746,SRR2340143,511145,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1958851700,39177034,None,50.0
2,SRP063562,SRX1219747,SRR2340144,511145,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,918302050,18366041,None,50.0
3,SRP063562,SRX1219748,SRR2340146,511145,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1430785650,28615713,None,50.0
4,SRP063562,SRX1219749,SRR2340147,511145,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1520846350,30416927,None,50.0


In [28]:
srp = 'SRP098789'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 26/26 [00:00<00:00, 109.95it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/hg38_SRP098789.py


In [27]:
df

,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP098789,SRX2536403,SRR5227288,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2104142750,42082855,None,50.0
1,SRP098789,SRX2536404,SRR5227289,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2082873050,41657461,None,50.0
2,SRP098789,SRX2536405,SRR5227290,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2023148650,40462973,None,50.0
3,SRP098789,SRX2536406,SRR5227291,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2057165950,41143319,None,50.0
4,SRP098789,SRX2536407,SRR5227292,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,3027621850,60552437,None,50.0
5,SRP098789,SRX2536408,SRR5227293,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2135456900,42709138,None,50.0
6,SRP098789,SRX2536409,SRR5227294,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,3800106100,76002122,None,50.0
7,SRP098789,SRX2536410,SRR5227295,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2306857400,46137148,None,50.0
8,SRP098789,SRX2536411,SRR5227296,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2636889200,52737784,None,50.0
9,SRP098789,SRX2536412,SRR5227297,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,3019068250,60381365,None,50.0


In [30]:
srp = 'SRP017942'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 11/11 [00:00<00:00, 33.19it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/hg38_SRP017942.py
Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP017942.py


In [37]:
srp = 'SRP045475'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df

100%|██████████| 6/6 [00:00<00:00, 251.68it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/BDGP6_SRP045475.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP045475,SRX679371,SRR1548659,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,9481573800,189631476,None,50.0
1,SRP045475,SRX679370,SRR1548658,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,718697100,14092100,None,51.0
2,SRP045475,SRX679368,SRR1548656,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,9997319700,99973197,None,100.0
3,SRP045475,SRX679369,SRR1548657,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,6723088500,67230885,None,100.0
4,SRP045475,SRX679372,SRR1548660,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,373767700,3737677,None,100.0
5,SRP045475,SRX679372,SRR1548661,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,868997500,8689975,None,100.0


In [41]:
srp = 'SRP108999'
df = db.sra_convert(srp)
df['taxon_id'] = 7227
df['library_strategy'] = 'SINGLE'
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 21/21 [00:00<00:00, 51.63it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/BDGP6_SRP108999.py


In [42]:
df

,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP108999,SRX2902866,SRR5667278,7227,None,None,SINGLE,None,None,676573468,20752434,None,32.602126
1,SRP108999,SRX2902865,SRR5667277,7227,None,None,SINGLE,None,None,2610438767,78959629,None,33.060423
2,SRP108999,SRX2902871,SRR5667283,7227,None,None,SINGLE,None,None,1104998701,32872272,None,33.614917
3,SRP108999,SRX2902864,SRR5667276,7227,None,None,SINGLE,None,None,1194135764,35199688,None,33.924612
4,SRP108999,SRX2902857,SRR5667269,7227,None,None,SINGLE,None,None,1539402070,45156367,None,34.090477
5,SRP108999,SRX2902856,SRR5667268,7227,None,None,SINGLE,None,None,4216803822,122734505,None,34.357118
6,SRP108999,SRX2902870,SRR5667282,7227,None,None,SINGLE,None,None,1940386794,56343352,None,34.438611
7,SRP108999,SRX2902872,SRR5667284,7227,None,None,SINGLE,None,None,2392506309,69053087,None,34.647348
8,SRP108999,SRX2902855,SRR5667267,7227,None,None,SINGLE,None,None,1621851282,46651854,None,34.764991
9,SRP108999,SRX2902861,SRR5667273,7227,None,None,SINGLE,None,None,1408728811,20062370,None,70.217467
